# 2. Feature Pipeline

## 2.1. Environment Setup
Handle repository cloning, dependency installation, and set up Python path.

In [ ]:
import sys
from pathlib import Path
import hopsworks
import warnings

warnings.filterwarnings("ignore", module="IPython")

def clone_repository() -> None:
    repo_dir = Path("pm25-forecast-openmeteo-aqicn")
    if repo_dir.exists():
        print(f"Repository already exists at {repo_dir.absolute()}")
        %cd pm25-forecast-openmeteo-aqicn
    else:
        print("Cloning repository...")
        !git clone https://github.com/KristinaPalmquist/pm25-forecast-openmeteo-aqicn.git
        %cd pm25-forecast-openmeteo-aqicn

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml


root_dir = Path().absolute()
for folder in ("src", "airquality", "notebooks"):
    if root_dir.parts[-1:] == (folder,):
        root_dir = Path(*root_dir.parts[:-1])
root_dir = str(root_dir)

if root_dir not in sys.path:
    sys.path.append(root_dir)

from utils import config

settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")
HOPSWORKS_API_KEY = settings.HOPSWORKS_API_KEY.get_secret_value()
project = hopsworks.login(engine="python", api_key_value=HOPSWORKS_API_KEY)
fs = project.get_feature_store()

## 2.2. Imports

In [ ]:
import datetime
import pandas as pd
import hopsworks
import json
import warnings
import requests
from utils import airquality
from dotenv import load_dotenv

warnings.filterwarnings("ignore")
load_dotenv()

## 2.3. Setup
Hopsworks and feature store setup - configure Hopsworks connection, retrieve API keys, and connect to existing air quality and weather feature groups.

In [ ]:
secrets = hopsworks.get_secrets_api()
AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value

today = datetime.date.today()

# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name="air_quality_all",
    version=1,
)

weather_fg = fs.get_feature_group(
    name="weather_all",
    version=1,
)

## 2.4. Sensor Location Loading
Retrieve sensor location data from Hopsworks secrets for all sensors and parse JSON location metadata.

In [ ]:
# Read all individual secrets for all sensors
all_secrets = secrets.get_secrets()
locations = {}
for secret in all_secrets:
    if secret.name.startswith("SENSOR_LOCATION_JSON_"):
        sensor_id = secret.name.replace("SENSOR_LOCATION_JSON_", "")
        location_str = secrets.get_secret(secret.name).value
        if location_str:
            locations[sensor_id] = json.loads(location_str)

In [ ]:
# Convert @ URLs to A URLs for Swedish sensors (AQICN API change)
fixed_count = 0
for sensor_id, location in locations.items():
    if "@" in location["aqicn_url"]:
        old_url = location["aqicn_url"]
        new_url = old_url.replace("/@", "/A")
        location["aqicn_url"] = new_url
        fixed_count += 1

if fixed_count > 0:
    print(f"🔧 Fixed {fixed_count} sensor URLs from @ to A format")
else:
    print("ℹ️ All sensor URLs already in correct format")

## 2.5. Helper Methods
Data processing functions - get daily weather forecasts and fetch current data, air quality and weather, for each sensor location.

In [ ]:
def get_daily_weather_forecast(city, latitude, longitude):
    hourly_df = airquality.get_hourly_weather_forecast(city, latitude, longitude)
    hourly_df = hourly_df.set_index("date")
    daily_df = hourly_df.between_time("11:59", "12:01")
    daily_df = daily_df.reset_index()


    daily_df["date"] = pd.to_datetime(daily_df["date"]).dt.date
    daily_df["date"] = pd.to_datetime(daily_df["date"])
    daily_df["city"] = city
    return daily_df


def fetch_data_for_location(location):
    country = location["country"]
    city = location["city"]
    street = location["street"]
    aqicn_url = location["aqicn_url"]
    latitude = location["latitude"]
    longitude = location["longitude"]

    aq_today_df = airquality.get_pm25(aqicn_url, country, city, street, today, AQICN_API_KEY)
    daily_df = get_daily_weather_forecast(city, latitude, longitude)
    return aq_today_df, daily_df

## 2.6. Data Collection
Loop through all sensors to fetch today's air quality data and weather forecasts, format data to match feature group schemas.

In [ ]:
aqs = []
weathers = []
print(f"🔍 Processing {len(locations)} sensor locations...")

for sensor, location in locations.items():
    try:
        aq_today_df, weather_daily_forecast_df = fetch_data_for_location(location)

        aq_today_df = aq_today_df.assign(
            sensor_id=str(sensor),
            street=location["street"],
            city=location["city"],
            country=location["country"],
            feed_url=location["aqicn_url"],
        )
        aq_today_df["date"] = pd.to_datetime(aq_today_df["date"])

        # Weather FG shape
        weather_daily_forecast_df = weather_daily_forecast_df.assign(
            sensor_id=str(sensor),
            city=location["city"],
            latitude=location["latitude"],
            longitude=location["longitude"],
        )
        weather_daily_forecast_df["date"] = pd.to_datetime(
            weather_daily_forecast_df["date"]
        )

        aqs.append(aq_today_df)
        weathers.append(weather_daily_forecast_df)
        
    except requests.exceptions.RequestException as e:
        print(f"⚠️  Skipping sensor {sensor}: {e}")
        continue
    except Exception as e:
        print(f"⚠️  Unexpected error with sensor {sensor}: {type(e).__name__}: {e}")
        continue

In [ ]:
print(weathers)

In [ ]:
aq_df = pd.concat(aqs) if aqs else pd.DataFrame()
if not aq_df.empty:
    aq_df["pm25"] = pd.to_numeric(aq_df["pm25"], errors="coerce").astype("float64")
    aq_df["date"] = pd.to_datetime(aq_df["date"]).dt.tz_localize(None)
    aq_df = aq_df.drop(columns=["url"], errors="ignore")

    # Data quality check 1: Remove rows with missing PM2.5 values
    initial_count = len(aq_df)
    aq_df = aq_df.dropna(subset=['pm25'])
    if len(aq_df) < initial_count:
        print(f"🧹 Removed {initial_count - len(aq_df)} rows with missing PM2.5 values")

# Get historical data for rolling window and lagged features
historical_start = today - datetime.timedelta(days=4)
historical_df = pd.DataFrame()

# Read historical data from feature group and filter for the last 4 days
try:
    historical_df = air_quality_fg.read()
    if not historical_df.empty:
        historical_df["date"] = pd.to_datetime(historical_df["date"]).dt.tz_localize(None)
        historical_df = historical_df[
            (historical_df["date"].dt.date >= historical_start) & (historical_df["date"].dt.date < today)
        ][["date", "sensor_id", "pm25"]]
except Exception:
    pass

In [ ]:
print(historical_df['date'])

In [ ]:
combined_df = pd.concat([historical_df, aq_df], ignore_index=True) if not historical_df.empty else aq_df
if not combined_df.empty:
    combined_df = airquality.add_rolling_window_feature(combined_df, window_days=3, column="pm25", new_column="pm25_rolling_3d")
    combined_df = airquality.add_lagged_features(combined_df, column="pm25", lags=[1, 2, 3])
    combined_df = airquality.add_nearby_sensor_feature(combined_df, locations, column="pm25_lag_1d", n_closest=3)
    
    # Data quality check 2: Clean up NaNs created by feature engineering
    before_cleaning = len(combined_df[combined_df["date"].dt.date == today])
    
    # Only keep today's data and remove rows where essential features are NaN
    aq_df = combined_df[combined_df["date"].dt.date == today].copy()
    
    # Remove rows where pm25 is still NaN after all processing
    aq_df = aq_df.dropna(subset=['pm25'])
    
    after_cleaning = len(aq_df)
    if before_cleaning > after_cleaning:
        print(f"🧹 Removed {before_cleaning - after_cleaning} rows with NaN values after feature engineering")
    
    print(f"📊 Final data quality: {len(aq_df)} clean rows ready for feature store")
else:
    aq_df = pd.DataFrame()
    print("⚠️  No data available for processing")
aq_df.head()

In [ ]:
weather_df = pd.concat(weathers) if weathers else pd.DataFrame()
if not weather_df.empty:
    weather_df["date"] = pd.to_datetime(weather_df["date"])
    
    # Data quality check 3: Remove rows with missing weather data
    initial_weather_count = len(weather_df)
    weather_df = weather_df.dropna(subset=['temperature_2m_mean', 'precipitation_sum', 'wind_speed_10m_max'])
    
    # Convert to float32 to match Hopsworks feature group schema
    weather_df["temperature_2m_mean"] = weather_df["temperature_2m_mean"].astype("float32")
    weather_df["precipitation_sum"] = weather_df["precipitation_sum"].astype("float32")
    weather_df["wind_speed_10m_max"] = weather_df["wind_speed_10m_max"].astype("float32")
    weather_df["wind_direction_10m_dominant"] = weather_df["wind_direction_10m_dominant"].astype("float32")
    
    if len(weather_df) < initial_weather_count:
        print(f"🧹 Removed {initial_weather_count - len(weather_df)} rows with missing weather data")
    
    print(f"🌤️  Weather data quality: {len(weather_df)} clean weather rows")
else:
    print("⚠️  No weather data available")
weather_df.head()

In [ ]:
print(weather_df['date'].unique())

In [ ]:
# Final validation before inserting to feature store
if not aq_df.empty and not weather_df.empty:
    print(f"✅ Inserting {len(aq_df)} air quality rows and {len(weather_df)} weather rows to feature store")
    air_quality_fg.insert(aq_df)
    weather_fg.insert(weather_df)
    print("📁 Data successfully inserted to feature store")
else:
    if aq_df.empty:
        print("⚠️  No clean air quality data to insert")
    if weather_df.empty:
        print("⚠️  No clean weather data to insert")

In [ ]:
print(weather_df['date'].unique())